In [ ]:
import matplotlib
#matplotlib.use('Agg') # this suppresses the console for plotting
import matplotlib.pyplot as plt

In [ ]:
import bz2
import numpy as np
from numpy import random, array
import pandas as pd
import os
import os.path
import pylab
import importlib
import imp
from importlib import reload
import gzip
import ntpath
from Bio import SeqIO
from glob import glob
from itertools import product
from functools import partial
from multiprocessing import Pool
import pickle
from scipy import interp
import argparse

In [ ]:
from sklearn.preprocessing import normalize, label_binarize, LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import shuffle
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import History, TensorBoard
from keras import backend as K
backend = K.backend()

In [ ]:
# import our private scripts
import load_spAB
import deep_learning_models
import plotting_utils_AEB
import stats_utils_AEB_110718
import config_file_AEB

species_directory = config_file_AEB.species_directory
data_directory = config_file_AEB.data_directory
analysis_directory = config_file_AEB.analysis_directory  
scripts_directory = config_file_AEB.scripts_directory 

In [ ]:
def run_model(data_set, norm_input, encoding_dim, encoded_activation, input_dropout_pct, dropout_pct, num_epochs, batch_size, n_splits, n_repeats, compute_informative_features, plot_iteration, graph_dir, outFile):
    
    # format strings for outputting the paramters associated with this run:
    summary_string, plotting_string= stats_utils_AEB_110718.format_input_parameters_printing(data_set, norm_input, encoding_dim, encoded_activation,input_dropout_pct,dropout_pct,num_epochs,batch_size,n_splits,n_repeats,compute_informative_features,plot_iteration)

    outFile_header='data_set\tnorm_input\tencoding_dim\tencoded_activation\tinput_dropout_pct\tdropout_pct\tnum_epochs\tbatch_size\tn_splits\tn_repeats\t'

    #################
    # Load the data # 
    #################
    print('Loading data...')

    data_normalized, labels, rskf = load_spAB.load_single_disease(data_set, n_splits, n_repeats, precomputed_kfolds=False)
    
    print("Dimensions of normalized species cnts: " + str(data_normalized.shape))
    print("Dimensions of labels: " + str(labels.shape))
    # rskf = repeated stratified k fold. This contains all the kfold-by-iteration combos. 


    ###################################################
    # iterate through the data kfolds and iterations #
    ###################################################

    # Create a dictionary to store the metrics of each fold 
    aggregated_statistics={} # key=n_repeat, values= dictionary with stats
    
    #needed to change datatype to list bc I was getting - TypeError: 'generator' object is not subscriptable
    rskf = list(rskf)

    for n_repeat in range(0,len(rskf[0])):
        
        print('Iteration %s...' %n_repeat)
        
        aggregated_statistics[n_repeat] = {}
        
        train_idx = rskf[0][n_repeat]
        test_idx = rskf[1][n_repeat]
        x_train, y_train = data_normalized[train_idx], labels[train_idx]
        x_test, y_test = data_normalized[test_idx], labels[test_idx]
    
        #standardize the data, mean=0, std=1
        if norm_input:
            x_train, x_test= stats_utils_AEB_110718.standardize_data(x_train, x_test)
    
        ###########################################
        # set up a model (supervised learning)    #
        ###########################################
        # note that the model has to be instantiated each time a new fold is started otherwise the weights will not start from scratch. 
    
        input_dim=len(data_normalized[0]) # this is the number of input kmers

        model=deep_learning_models.create_supervised_model(input_dim, encoding_dim, encoded_activation,input_dropout_pct, dropout_pct)
    
        #weightFile = os.environ['HOME'] + '/deep_learning_microbiome/data/weights.txt'
       
        ##################################################
        # Fit the model with the train data of this fold #
        ##################################################
        history = History()
        # history is a dictionary. To get the keys, type print(history.history.keys())
        
        model.fit(x_train, y_train, 
                  epochs=num_epochs, 
                  batch_size=len(x_train), 
                  shuffle=True,
                  validation_data=(x_test, y_test),
                  verbose=0,
                  callbacks=[history])
    
        # predict using the held out data
        y_pred=model.predict(x_test)
        
        # save the weights of this model. TODO 
    
        ################################################################
        # Compute summary statistics                                   #
        ################################################################
        # Store the results of this fold in aggregated_statistics
        aggregated_statistics = stats_utils_AEB_110718.compute_summary_statistics(y_test, y_pred, history, aggregated_statistics, n_repeat)

        # could  plot everything (roc, accuracy vs epoch, loss vs epoch, confusion matrix, precision recall) for each fold, but this will produce a lot of graphs. 
        if compute_informative_features:
            shap_values, shap_values_summed = stats_utils_AEB_110718.compute_shap_values_deeplearning(input_dim, model, x_test)
            aggregated_statistics[n_repeat]['shap_values_summed']=shap_values_summed
            aggregated_statistics[n_repeat]['shap_values']=shap_values

        # also plot:
        #shap.summary_plot(shap_values, X, plot_type="bar")
        #shap.summary_plot(shap_values, X)

    ##############################################
    # aggregate the results from all the k-folds #
    # Print and Plot                             #
    ##############################################
    print('Aggregating statistics across iterations and printing/plotting...')

    stats_utils_AEB_110718.aggregate_statistics_across_folds(aggregated_statistics, rskf, n_splits, outFile, summary_string, plotting_string, outFile_header)


    ###################
    # Aggregate shap: #
    ###################

    if compute_informative_features: 
        print('Computing informative features with Shap...')
        stats_utils_AEB_110718.aggregate_shap(aggregated_statistics, rskf)


    #####################################
    # TSNE visualization                #
    # Annamarie                         #
    # find the weights of the best fold #
    #####################################




##############################
# parser for the config dict #
##############################
def parse_config_and_run(config_dict, outFile):
    data_sets=config_dict['data_set']
    #kmer_sizes=config_dict['kmer_size']
    norm_inputs=config_dict['norm_input']
    encoding_dims=config_dict['encoding_dim']
    encoded_activations=config_dict['encoded_activation']
    input_dropout_pcts=config_dict['input_dropout_pct']
    dropout_pcts=config_dict['dropout_pct'] 
    num_epochss=config_dict['num_epochs']
    batch_sizes=config_dict['batch_size']
    n_splitss=config_dict['n_splits']
    n_repeatss=config_dict['n_repeats']
    compute_informative_featuress=config_dict['compute_informative_features']
    plot_iterations=config_dict['plot_iteration'] 
    graph_dirs=config_dict['graph_dir'] 

    for data_set in data_sets:
        #for kmer_size in kmer_sizes:
        for norm_input in norm_inputs:
            for encoding_dim in encoding_dims:
                for encoded_activation in encoded_activations:
                    for input_dropout_pct in input_dropout_pcts:
                        for dropout_pct in dropout_pcts:
                            for num_epochs in num_epochss:
                                for batch_size in batch_sizes:
                                    for n_splits in n_splitss:
                                        for n_repeats in n_repeatss:
                                            for compute_informative_features in compute_informative_featuress:
                                                for plot_iteration in plot_iterations:
                                                    for graph_dir in graph_dirs:
                                                        
                                                        run_model(data_set, 
                                                                      #kmer_size,
                                                                      norm_input,
                                                                      encoding_dim,
                                                                      encoded_activation,
                                                                      input_dropout_pct,
                                                                      dropout_pct,
                                                                      num_epochs,
                                                                      batch_size,
                                                                      n_splits,
                                                                      n_repeats,
                                                                      compute_informative_features,
                                                                      plot_iteration,
                                                                      graph_dir, 
                                                                      outFile)

In [ ]:
# making sure loader is working

data_set = ['MetaHIT']
species_cnts, labelz, feats = load_spAB.load_species(data_set)
print("LOADED DATASET " + str(data_set) + ": " + str(len(species_cnts)) + " SAMPLES")

# Checking the data
np.count_nonzero(species_cnts==0, axis = 1) # a lot of nonzeros
np.any(np.isnan(species_cnts)) # returns False
np.all(np.isfinite(species_cnts)) # returns True

In [ ]:
feats

In [ ]:
config_dict=config_file_AEB.config
#for data_set in data_sets_to_use:
parse_config_and_run(config_dict, outFile="summary_statistics_11718.txt")

In [ ]:
data_set='MetaHIT'
print('Loading data...')

data_normalized, labels, rskf = load_spAB.load_single_disease(data_set, n_splits=10, n_repeats=5, precomputed_kfolds=False)
#data_normalized = data_normalized[:, ~np.all(data_normalized == 0, axis=0)]    
print("Dimensions of normalized species cnts: " + str(data_normalized.shape))
print("Dimensions of labels: " + str(labels.shape))

rskf = list(rskf)

for n_repeat in range(0,len(rskf[0])):
    train_idx = rskf[0][n_repeat]
    test_idx = rskf[1][n_repeat]
    x_train, y_train = data_normalized[train_idx], labels[train_idx]
    x_test, y_test = data_normalized[test_idx], labels[test_idx]
    
    #standardize the data, mean=0, std=1
    #x_train, x_test= stats_utils_AEB_110718.standardize_data(x_train, x_test)

In [ ]:
data_set=['MetaHIT']
allowed_labels=['0','1']
species_cnts, labels, features = load_spAB.load_species(data_set)

In [ ]:
# get rid of features that are zero for all of the samples
species_cnts = species_cnts[:, ~np.all(species_cnts == 0, axis=0)]


labels=np.asarray(labels)
labels=labels.astype(np.int)

data_normalized = species_cnts

In [ ]:
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=1)

In [ ]:
rskf = rskf.split(data_normalized, labels)

In [ ]:
list(rskf)

In [ ]:
rskf = list(rskf)

In [ ]:
range(0,len(rskf[0]))

In [ ]:
for n_repeat in range(0,len(rskf[0])):
    train_idx = rskf[0][n_repeat]
    test_idx = rskf[1][n_repeat]
    x_train, y_train = data_normalized[train_idx], labels[train_idx]
    x_test, y_test = data_normalized[test_idx], labels[test_idx]
    
    sample_mean = x_train.mean(axis=0)
    sample_std = x_train.std(axis=0)
    
    #x_train = (x_train - sample_mean) / sample_std
    
    #x_test = (x_test - sample_mean) / sample_std

In [ ]:
rskf[1]

In [ ]:
rskf[0][0]

In [ ]:
rskf[1][0]

In [ ]:
#test first repeat
len(rskf[1][0])

In [ ]:
np.count_nonzero(x_train==0, axis=0)

In [ ]:
x_train.shape

In [ ]:
np.count_nonzero(sample_mean==0, axis=0)

In [ ]:
np.count_nonzero(sample_std==0, axis=0)

# Plots for blog post

In [ ]:
data_set=['MetaHIT']
allowed_labels=['0','1']
species_cnts, labels, features = load_spAB.load_species(data_set)
    
# get rid of features that are zero for a fraction of the samples
species_cnts = species_cnts[:, ~np.all(species_cnts == 0, axis=0)]

labels=np.asarray(labels)
labels=labels.astype(np.int)

data_normalized = species_cnts

rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=5)
rskf = rskf.split(data_normalized, labels)
rskf = list(rskf)

for n_repeat in range(0,len(rskf[0])):
    train_idx = rskf[0][n_repeat]
    test_idx = rskf[1][n_repeat]
    x_train, y_train = data_normalized[train_idx], labels[train_idx]
    x_test, y_test = data_normalized[test_idx], labels[test_idx]
    
    #x_train, x_test = stats_utils_AEB_110718.standardize_data(x_train, x_test)

In [ ]:
data_normalized.shape

In [ ]:
zeros = list(np.count_nonzero(data_normalized==0, axis=0))

In [ ]:
prevs = np.count_nonzero(data_normalized==0, axis=0)

prevs2 = [i for i in zeros if i >= 100]

In [ ]:
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image
%config InlineBackend.figure_format = 'svg'

In [ ]:
# Plotting hist without kde
ax = sns.distplot(prevs, kde=False)
# Labels
plt.title("MetaHIT species data")
plt.xlabel("Number of zero entries\n(note: there are 110 samples)")
plt.ylabel("Taxa (i.e. column) count")

# plot the zoomed portion
sub_axes1 = plt.axes([.25, .5, .38, .3]) 
sns.distplot(prevs2,kde=False)

plt.show()

In [ ]:
print(np.any(np.isnan(x_train))) # returns False
print(np.all(np.isfinite(x_train))) # returns True

print(np.any(np.isnan(x_test))) # returns False
print(np.all(np.isfinite(x_test))) # returns True

In [ ]:
x_train, x_test = stats_utils_AEB_110718.standardize_data(x_train, x_test)

In [ ]:
covs = data_normalized.std(axis=0) / data_normalized.mean(axis=0)

In [ ]:
covs.shape

In [ ]:
sns.distplot(covs, bins=20, kde=False)
plt.title("Coefficient of Variation\n(MetaHIT dataset)")
plt.xlabel("COV score")
plt.ylabel("No. features (taxa)")
plt.show()